In [2]:
#Shucen Liu
#Assignment 2
#CMSC 25025

from pyspark.sql import SparkSession
from pyspark.sql import Row
import os
import re
import string
from collections import Counter
import math
import numpy as np
import json
from pprint import pprint
import re


spark  = SparkSession.builder.master('local').appName('sou').getOrCreate()
df = spark.read.json('/project/cmsc25025/sou/speeches.json')


In [6]:
def collect_words(s):
    '''
    Convert the speech to a collection of words
    '''
    s = s.lower().encode('utf-8').translate(string.maketrans("",""), string.punctuation)
    return s.split()

speech = df.rdd.map(lambda x: (x['president'], x['year'], collect_words(x['text'])))
words = speech.map(lambda x: x[2]).reduce(lambda a, b: a + b)
words_pool = Counter(words).items()
words_pool.sort(key = lambda item: item[1])

def find_terms():
    '''
    Given the word pool, find the terms that appear frequently in the documents 
    (except the most common 20 words)
    '''
    terms = []
    for word in words_pool[:(len(words_pool) - 20)]:
        if word[1] > 50:
            terms.append(word[0])
    return terms

TERMS = find_terms() #The vocabulary that I will use for this assignment

In [7]:
from pyspark.sql import Row
#(a)
def find_frequency(word, word_list):
    if word not in word_list:
        num_appear = (0, 0) 
    else:
        num_appear = (word_list.count(word), 1)
    return num_appear

def find_num_doc(word):
    num_doc = speech.map(lambda x: whether_exist(word, x[2])).reduce(lambda a, b: a + b)
    return num_doc
TOTAL = speech.map(lambda x: 1).reduce(lambda a, b: a + b)
def find_vector(word_list):
    freq_list = []
    existence_ls = []
    for word in TERMS:
        frequency = find_frequency(word, word_list)
        freq = float(frequency[0])
        existence = float(frequency[1])
        freq_list.append(freq)
        existence_ls.append(existence)
    return freq_list, existence_ls
speech_vector_unadj = speech.map(lambda x: (x[0], x[1], find_vector(x[2])[0], find_vector(x[2])[1]))
num_appear = speech_vector_unadj.map(lambda x: np.array(x[3])).reduce(lambda x, y: x + y)
vector_adj = speech_vector_unadj.map(lambda x: Row(president = x[0], year = x[1], 
                                    vector = np.array(x[2]) * np.log(TOTAL * np.reciprocal(num_appear))))

In [10]:
#(b)
pairs = vector_adj.cartesian(vector_adj)
def similar_score(x, y):
    score = np.dot(x, y)/(np.linalg.norm(x) * np.linalg.norm(y))
    return score
similarity = pairs.map(lambda x: ((x[0]['president'], x[0]['year']), (x[1]['president'], x[1]['year']), 
                                  similar_score(x[0]['vector'], x[1]['vector'])))
dif_pres = similarity.filter(lambda x: x[0][0] != x[1][0])
same_pres = similarity.filter(lambda x: x[0][0] == x[1][0])
pres_pairs = dif_pres.map(lambda x: ((x[0], x[1]), (x[2], 1))).reduceByKey(
                        lambda a, b: (a[0] + b[0], a[1] + b[1])).mapValues(lambda v: v[0]/v[1])

In [ ]:
#(b)
print "They do not seem very similar to me, I think we also need to consider the order of the words, if we want to have a better similarity measure."
similar_dif = dif_pres.sortBy(lambda x: x[2], ascending=False).map(lambda x: (x[0], x[1])).take(50)
similar_same = same_pres.sortBy(lambda x: x[2], ascending=False).map(lambda x: (x[0], x[1])).take(50)
similar_pres = pres_pairs.sortBy(lambda x: x[1], ascending=False).map(lambda x: x[0]).take(25)

In [ ]:
#(c)
from pyspark.mllib.clustering import KMeans
print "The clusters kind of make sense, in that same presidents usually lie in the same cluster"
clusters = KMeans.train(vector_adj.map(lambda x: x['vector']), 10, maxIterations=50,
initializationMode="random")
df_clustered = vector_adj.rdd.map(lambda x: Row(cluster = clusters.predict(x['vector']), president=x['president'], 
                                                year=x['year']))
clusteredschema = spark.createDataFrame(df_clustered)
clusteredschema.createOrReplaceTempView("clustered_data")
for i in range(0, 10):
    query = str("SELECT cluster, label FROM clustered_data WHERE cluster = " + str(i))
    cluster = spark.sql(query)
    cluster.show()